# Understanding the way to use pre-trained model or transfer learning

In [1]:
# importing required libraries

from keras.models import Sequential
from scipy.misc import imread
get_ipython().magic('matplotlib inline')
import matplotlib.pyplot as plt
import numpy as np
import keras
from keras.layers import Dense
import pandas as pd

C:\Users\Ankit\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [46]:
#import better_exceptions
from keras.applications import ResNet50, InceptionResNetV2, VGG16
from keras.layers import Dense
from keras.models import Model
from keras import backend as K
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD, Adam

In [20]:
def age_mae(y_true, y_pred):
    true_age = K.sum(y_true * K.arange(0, 101, dtype="float32"), axis=-1)
    pred_age = K.sum(y_pred * K.arange(0, 101, dtype="float32"), axis=-1)
    mae = K.mean(K.abs(true_age - pred_age))
    return mae

In [43]:
def get_model(model_name="ResNet50"):
    base_model = None

    if model_name == "ResNet50":
        #base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3), pooling="avg")
        base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(48, 48, 3), pooling="avg")
    elif model_name == "InceptionResNetV2":
        #base_model = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=(299, 299, 3), pooling="avg")
        base_model = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=(48, 48, 3), pooling="avg")
    elif model_name == "VGG16":
        #base_model = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3), pooling="avg")
        base_model = VGG16(include_top=False, weights='imagenet', input_shape=(48, 48, 3), pooling="avg")

    prediction = Dense(units=101, kernel_initializer="he_normal", use_bias=False, activation="softmax",
                       name="pred_age")(base_model.output)

    model = Model(inputs=base_model.input, outputs=prediction)

    return model

In [22]:
def get_optimizer(opt_name, lr):
    if opt_name == "sgd":
        return SGD(lr=lr, momentum=0.9, nesterov=True)
    elif opt_name == "adam":
        return Adam(lr=lr)
    else:
        raise ValueError("optimizer name should be 'sgd' or 'adam'")


In [23]:
def create_model(model_name, opt_name, lr):
    model = get_model(model_name=model_name)
    opt = get_optimizer(opt_name, lr)
    model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=[age_mae])
    #model.summary()
    return model

In [24]:
def train_model(model_name, opt_name, lr, train_x, train_y, val_x, val_y, epochs, batch_size):
    model = create_model(model_name, opt_name, lr)
    model.fit(train_x, train_y,
                    epochs=epochs,
                    batch_size=batch_size, 
                    validation_data=(val_x, val_y))
    return model

In [25]:
def predict(model, test_x):
    for i in test_x:
        #update this function properly
        pred_y = model.predict(test_x)
    return pred_y

In [2]:
if ('trainingFaces' not in globals()):  # In ipython, use "run -i homework2_template.py" to avoid re-loading of data
        trainingFaces = np.load("facesAndAges/faces.npy")
        trainingLabels = np.load("facesAndAges/ages.npy")
        #testingFaces = np.load("data/mnist_validation_images.npy")
        #testingLabels = np.load("data/mnist_validation_labels.npy")

In [5]:
trainingFaces.shape

(7500, 48, 48)

In [6]:
trainingLabels.shape


(7500,)

In [8]:
np.min(trainingLabels)

0.0

In [15]:
trainX,testX, trainY, testY = train_test_split(trainingFaces, trainingLabels, test_size=0.1, random_state=42)

In [16]:
#trainX.shape



(6750, 48, 48)

In [30]:
train_x, val_x, train_y, val_y = train_test_split(trainX, trainY, test_size=0.2, random_state=42)

In [39]:
#train_x.shape
train_x_2 = np.stack([train_x]*3, axis=-1)
val_x_2 = np.stack([val_x]*3, axis=-1)
train_y_2 = to_categorical(train_y, 101)
val_y_2 = to_categorical(val_y, 101)

In [41]:
#val_y_2[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [ ]:
model = train_model('ResNet50', 'adam', 1e-3, train_x_2, train_y_2, val_x_2, val_y_2, 40, 100)

Instructions for updating:
Use tf.cast instead.
Train on 5400 samples, validate on 1350 samples
Epoch 1/40
5400/5400 [==============================] - 520s 96ms/step - loss: 4.6939 - age_mae: 13.8931 - val_loss: 4.0107 - val_age_mae: 11.4704
Epoch 2/40
5400/5400 [==============================] - 517s 96ms/step - loss: 4.1027 - age_mae: 11.8972 - val_loss: 12.0542 - val_age_mae: 13.1998
Epoch 3/40
5400/5400 [==============================] - 521s 97ms/step - loss: 3.8976 - age_mae: 11.0572 - val_loss: 7.6803 - val_age_mae: 12.1231
Epoch 4/40
5400/5400 [==============================] - 521s 96ms/step - loss: 3.7829 - age_mae: 10.1752 - val_loss: 4.3823 - val_age_mae: 10.4409
Epoch 5/40
5400/5400 [==============================] - 518s 96ms/step - loss: 3.5979 - age_mae: 9.3306 - val_loss: 11.5829 - val_age_mae: 11.9297
Epoch 6/40
5000/5400 [==========================>...] - ETA: 36s - loss: 3.7879 - age_mae: 9.8960

In [33]:
train_x_2 = np.stack([train_x]*3, axis=-1)

In [34]:
train_x.shape

(5400, 48, 48)

In [38]:
train_x[:5]

array([[[ 52.,  46.,  47., ...,  57.,  57.,  56.],
        [ 52.,  45.,  47., ...,  53.,  62.,  58.],
        [ 51.,  46.,  46., ...,  59.,  62.,  53.],
        ...,
        [ 37.,  35.,  30., ..., 111., 181., 240.],
        [ 41.,  40.,  34., ..., 135., 202., 249.],
        [ 42.,  44.,  39., ..., 160., 220., 253.]],

       [[ 43.,  46.,  40., ...,  28.,  26.,  26.],
        [ 43.,  42.,  37., ...,  37.,  31.,  28.],
        [ 40.,  38.,  36., ...,  37.,  31.,  30.],
        ...,
        [ 40.,  33.,  28., ...,  10.,  13.,  17.],
        [ 44.,  40.,  31., ...,  14.,  15.,  17.],
        [ 31.,  40.,  44., ...,  13.,  13.,  11.]],

       [[  9.,   4.,  28., ...,  59.,  45.,  26.],
        [  7.,  19.,  33., ...,  67.,  26.,  23.],
        [ 16.,  28.,  41., ...,  62.,  32.,  19.],
        ...,
        [220., 216., 199., ..., 192., 199., 172.],
        [224., 220., 214., ..., 195., 190., 115.],
        [222., 222., 222., ..., 196., 147., 167.]],

       [[239., 192.,  29., ...,  19.,

In [36]:
train_x_2


array([[[[ 52.,  52.,  52.],
         [ 46.,  46.,  46.],
         [ 47.,  47.,  47.],
         ...,
         [ 57.,  57.,  57.],
         [ 57.,  57.,  57.],
         [ 56.,  56.,  56.]],

        [[ 52.,  52.,  52.],
         [ 45.,  45.,  45.],
         [ 47.,  47.,  47.],
         ...,
         [ 53.,  53.,  53.],
         [ 62.,  62.,  62.],
         [ 58.,  58.,  58.]],

        [[ 51.,  51.,  51.],
         [ 46.,  46.,  46.],
         [ 46.,  46.,  46.],
         ...,
         [ 59.,  59.,  59.],
         [ 62.,  62.,  62.],
         [ 53.,  53.,  53.]],

        ...,

        [[ 37.,  37.,  37.],
         [ 35.,  35.,  35.],
         [ 30.,  30.,  30.],
         ...,
         [111., 111., 111.],
         [181., 181., 181.],
         [240., 240., 240.]],

        [[ 41.,  41.,  41.],
         [ 40.,  40.,  40.],
         [ 34.,  34.,  34.],
         ...,
         [135., 135., 135.],
         [202., 202., 202.],
         [249., 249., 249.]],

        [[ 42.,  42.,  42.],
       